# Таблицы DataFrame со сложными комбинациями подписей осей.

# *Pandas MultiIndex*

### Автор: Шабанов П.А.

#### URL: https://progeoru.blogspot.com
01/10/2018

<a id='up'></a>
### Цель

+ Научиться эффективно пользоваться мультииндексами pandas.DataFrames

### План

1. [Мультииндексы DataFrame](#multi)

2. [Как эффективно брать срезы от мульииндексированных DataFrames](#multiSlices)
    + [df.xs](#xs);
    + [pd.IndexSlice](#idx).

<a id='multi'></a>
## Мультииндексы DataFrame
[Вверх](#up)

**DataFrames** из модуля **pandas** поддерживают `мультииндексирование`, т.е. группирование подписей индексов строк или названий столбцов. 

Иногда данные строк/стобцов DataFrame имеют составные подписи. Например, A1, A2, A3, B1, B2, B3. Очень разумным кажется группировка в духе словаря {A:[1, 2, 3], B:[1, 2, 3]}. При такой группировке модно было бы обращаться как к "ключам", так и к общим значениям. 

Собственно мультииндекс - это составной индекс, представляющий собой обычно список списков. Число вложенных списков определяется количеством уровней группировок. В приведённом выше примере таких группировок две: 

> [['A', 'A', 'A', 'B', 'B', 'B'], [1, 2, 3, 1, 2, 3]]

Так для случаев двух уровней таблица с мульииндексированием названий столбцов будет выглядеть так:

In [1]:
#@author: pash

import numpy as np
import pandas as pd

NZ = 5
NY = 3
NX = 3

z = np.arange(2010, 2015)   # NZ
y = ['s1', 's2', 's3']   # NY
x = ['A', 'B', 'C']   # NX

NC = 4
columns = ['data1', 'data2', 'data3', 'data4']

arrC = (np.arange(NX*NY*NC)).reshape((NC, NX*NY))

mix2 = pd.MultiIndex.from_product([x, y], names=['dataset', 'param'])
df2 = pd.DataFrame(data=arrC, 
                  columns=mix2,
                  index=columns
                  )

df2

dataset   A           B           C        
param    s1  s2  s3  s1  s2  s3  s1  s2  s3
data1     0   1   2   3   4   5   6   7   8
data2     9  10  11  12  13  14  15  16  17
data3    18  19  20  21  22  23  24  25  26
data4    27  28  29  30  31  32  33  34  35

Для случая же мульииндексирования трёх уровней, но уже подписей строк - так:

In [2]:
#@author: pash

import numpy as np
import pandas as pd

NZ = 5
NY = 3
NX = 3

z = np.arange(2010, 2015)   # NZ
y = ['s1', 's2', 's3']   # NY
x = ['A', 'B', 'C']   # NX

NC = 4
columns = ['data1', 'data2', 'data3', 'data4']

arrR = (np.arange(NX*NY*NZ*NC)).reshape((NX*NY*NZ, NC))

mix3 = pd.MultiIndex.from_product([z, y, x], names=['years', 'param', 'dataset'])
df3 = pd.DataFrame(data=arrR,
                  columns=columns,
                  index=mix3
                  )

print(df3.head(9))
df3.tail(9)

                     data1  data2  data3  data4
years param dataset                            
2010  s1    A            0      1      2      3
            B            4      5      6      7
            C            8      9     10     11
      s2    A           12     13     14     15
            B           16     17     18     19
            C           20     21     22     23
      s3    A           24     25     26     27
            B           28     29     30     31
            C           32     33     34     35


data1  data2  data3  data4
years param dataset                            
2014  s1    A          144    145    146    147
            B          148    149    150    151
            C          152    153    154    155
      s2    A          156    157    158    159
            B          160    161    162    163
            C          164    165    166    167
      s3    A          168    169    170    171
            B          172    173    174    175
            C          176    177    178    179

<a id='multiSlices'></a>
## Как эффективно брать срезы от мульииндексированных DataFrames
[Вверх](#up)

Использование мультииндексов бессмысленно без поддержки более эффективной индексации данных, более понятного "slicing"-а.

Есть два хороших вариант.

1. **df.xs()** - подходит только для подписей строк.

Этот метод подходит для любых DataFrames, даже без мультииндексов.

In [3]:
#@author: pash

a = df3.xs('B', level='dataset')
a

data1  data2  data3  data4
years param                            
2010  s1         4      5      6      7
      s2        16     17     18     19
      s3        28     29     30     31
2011  s1        40     41     42     43
      s2        52     53     54     55
      s3        64     65     66     67
2012  s1        76     77     78     79
      s2        88     89     90     91
      s3       100    101    102    103
2013  s1       112    113    114    115
      s2       124    125    126    127
      s3       136    137    138    139
2014  s1       148    149    150    151
      s2       160    161    162    163
      s3       172    173    174    175

In [4]:
b = df3.xs(2010, level='years')
b

data1  data2  data3  data4
param dataset                            
s1    A            0      1      2      3
      B            4      5      6      7
      C            8      9     10     11
s2    A           12     13     14     15
      B           16     17     18     19
      C           20     21     22     23
s3    A           24     25     26     27
      B           28     29     30     31
      C           32     33     34     35

In [5]:
c = df3.xs('s2', level='param')
c

data1  data2  data3  data4
years dataset                            
2010  A           12     13     14     15
      B           16     17     18     19
      C           20     21     22     23
2011  A           48     49     50     51
      B           52     53     54     55
      C           56     57     58     59
2012  A           84     85     86     87
      B           88     89     90     91
      C           92     93     94     95
2013  A          120    121    122    123
      B          124    125    126    127
      C          128    129    130    131
2014  A          156    157    158    159
      B          160    161    162    163
      C          164    165    166    167

<a id='idx'></a>
2. **pd.IndexSlice** - удобный и понятный в идеологии pandas.DataFrame способ взятия срезов.

[Вверх](#up)

Следует помнить, что несмотря на составные подписи мультииндексов, объект DataFrame остаётся двумерным массивом с двумя осями. Синтаксис осуществляется через `df.loc`, где по каждой оси используется объект pd.IndexSlice. 

In [6]:
idx = pd.IndexSlice

a = df2.loc[idx[:], idx[:, 's1']]
print('a') 
a

a


dataset,A,B,C
param,s1,s1,s1
data1,0,3,6
data2,9,12,15
data3,18,21,24
data4,27,30,33


In [7]:
b = df3.loc[idx[:, 's1', 'A'], idx[:]]
print('b')
b

b


,,,data1,data2,data3,data4
years,param,dataset,,,,
2010,s1,A,0,1,2,3
2011,s1,A,36,37,38,39
2012,s1,A,72,73,74,75
2013,s1,A,108,109,110,111
2014,s1,A,144,145,146,147


[Вверх](#up)

### Автор: Шабанов П.А.

#### URL: https://progeoru.blogspot.com
01/10/2018